In [1]:
import Pkg
Pkg.add("MAT")
Pkg.add("Flux")
Pkg.add("NNlib")
Pkg.add("CUDA")
Pkg.add("Statistics")

using Flux
# using Flux: crossentropy, onecold, onehotbatch, train!, params 
using MAT
using CUDA
using Statistics
using Distributions, StatsPlots, Random

    Updating registry at `C:\Users\Nicol\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nicol\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Nicol\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nicol\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Nicol\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nicol\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Nicol\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nicol\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Nicol\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nicol\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Nicol\.julia\environments\v1.8\Manifest.toml`


In [9]:
#Cancer features - 29
begin
    Cancer_data_labels = ["Q3", "Q8", "Q12", "Q15", "Q17", "Q22", "Q24", "Q26", "Q28", "Q29","Q30","Q31", "Q32","Q33","Q36","Q39","Q41","Q44","Q45","Q47","Q48", "Q51","Q53","Q55","Q57","Q58","Q61","Q64","Q68"]
    Cancer_dict = Dict()
    for i in Cancer_data_labels
        file1 = matopen("/Users/Nicol/OneDrive/Desktop/Nicole_UNSW/VIP 2023/Features - Qatar/$i"*"_LESION_full_feature.mat")
        feature = read(file1, "full_feature")
        push!(Cancer_dict,"$i"=>reshape(feature,(160*160,15)))
    end
end 

#Precancerous features - 5
begin
    Precancerous_data_labels = ["Q18","Q19","Q23","Q25","Q60"]
    Precancerous_dict = Dict()
    for i in Precancerous_data_labels
        file1 = matopen("/Users/Nicol/OneDrive/Desktop/Nicole_UNSW/VIP 2023/Features - Qatar/$i"*"_LESION_full_feature.mat")
        feature = read(file1, "full_feature")
        push!(Precancerous_dict,"$i"=>reshape(feature,(160*160,15)))
    end
end

#Benign features - 35
begin
    Benign_data_labels = ["Q1","Q2","Q4","Q5","Q6","Q7","Q9","Q10","Q11","Q13","Q14","Q16","Q20","Q21", "Q27","Q34","Q35","Q37","Q38","Q40","Q42","Q43","Q46","Q49","Q50", "Q52","Q54","Q56","Q59","Q62","Q63","Q65","Q66","Q67","Q69"]
    Benign_dict = Dict()
    for i in Benign_data_labels
        file1 = matopen("/Users/Nicol/OneDrive/Desktop/Nicole_UNSW/VIP 2023/Features - Qatar/$i"*"_LESION_full_feature.mat")
        feature = read(file1, "full_feature")
        push!(Benign_dict,"$i"=>reshape(feature,(160*160,15)))
    end
end

In [10]:
cancer_testing = []
cancer_training = []

pre_cancer_testing = []
pre_cancer_training = []

benign_testing = []
benign_training = []

# Taking 20% for testing

for key in ["Q29", "Q17", "Q26", "Q61", "Q36", "Q3"]
    data = pop!(Cancer_dict, key)
    push!(cancer_testing, data') # making data (15,160*160) - features are rows
end
println(length(cancer_testing))
println(size(cancer_testing[1]))

for (key,value) in Cancer_dict
    data = pop!(Cancer_dict, key)
    push!(cancer_training, data')
end

for key in ["Q19"]
    data = pop!(Precancerous_dict, key)
    push!(pre_cancer_testing, data')
end

for (key,value) in Precancerous_dict
    data = pop!(Precancerous_dict, key)
    push!(pre_cancer_training, data')
end

for key in ["Q10", "Q20", "Q35", "Q43", "Q50", "Q56", "Q69"]
    data = pop!(Benign_dict, key)
    push!(benign_testing, data')
end

for (key,value) in Benign_dict
    data = pop!(Benign_dict, key)
    push!(benign_training, data')
end

6
(15, 25600)


In [11]:
cancer_testing = hcat(cancer_testing...)
cancer_training = hcat(cancer_training...)
println(length(cancer_testing))

pre_cancer_testing = hcat(pre_cancer_testing...)
pre_cancer_training = hcat(pre_cancer_training...)

combined_cancer_testing = hcat(cancer_testing, pre_cancer_testing)
combined_cancer_training = hcat(cancer_training, pre_cancer_training)

benign_testing = hcat(benign_testing...)
benign_training = hcat(benign_training...)

X = hcat(combined_cancer_training, benign_training)
println(size(X))
Y = vcat(ones(size(combined_cancer_training,2)),zeros(size(benign_training,2)))

X_test = hcat(combined_cancer_testing, benign_testing)
Y_test = vcat(ones(size(combined_cancer_testing,2)),zeros(size(benign_testing,2)))

2304000
(15, 1408000)


358400-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [12]:
model = Chain(
    Dense(15, 45, relu),
    BatchNorm(45, relu),
    Dense(45, 45, relu),
    BatchNorm(45, relu),
    Dense(45, 15, relu),
    BatchNorm(15, relu),
    sigmoid
)

# define loss function

loss(x, y) = Flux.crossentropy(model(x), y)

# track parameters

ps = Flux.params(model)

# select optimizer

learning_rate = 0.01

# Adaptive Moment Estimation
opt = ADAM(learning_rate)

Adam(0.01, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [13]:
# train model

data = Flux.Data.DataLoader((X, Y'), batchsize=100,shuffle=true);

loss_history = []

epochs = 50

for epoch in 1:epochs
    # train model
    
    Flux.train!(loss, ps, [(X, Y')], opt)
    
    # print report
    train_loss = loss(X, Y')
    push!(loss_history, train_loss)
    println("Epoch = $epoch : Training Loss = $train_loss")
end

Epoch = 1 : Training Loss = 0.1891127542855328
Epoch = 2 : Training Loss = 0.08490254308708162
Epoch = 3 : Training Loss = 0.039554598803528136
Epoch = 4 : Training Loss = 0.02041809263184984
Epoch = 5 : Training Loss = 0.013539416325197454
Epoch = 6 : Training Loss = 0.010650003724126562
Epoch = 7 : Training Loss = 0.008838777708123486
Epoch = 8 : Training Loss = 0.0073376607208944025
Epoch = 9 : Training Loss = 0.005984096590978413
Epoch = 10 : Training Loss = 0.004742119580730087
Epoch = 11 : Training Loss = 0.0036389657004457505
Epoch = 12 : Training Loss = 0.002656139155840518
Epoch = 13 : Training Loss = 0.0018671606665539037
Epoch = 14 : Training Loss = 0.0012453314508151063
Epoch = 15 : Training Loss = 0.0007949429939119095
Epoch = 16 : Training Loss = 0.00048406430086370676
Epoch = 17 : Training Loss = 0.00028367713598841825
Epoch = 18 : Training Loss = 0.0001614118379674597
Epoch = 19 : Training Loss = 9.001003770764087e-5
Epoch = 20 : Training Loss = 4.964282881184673e-5
Epo

In [14]:
# make predictions

y_hat = model(X_test)
y_hat = map(x -> x > 0.5 ? 1 : 0, y_hat)

# is the output a column or row vector
print("Accuracy: ")
print(Statistics.mean(y_hat .== Y_test'))

# display results

index = collect(1:length(Y_test))

check_display = [index y_hat' Y_test]

check_display

# Sensitivity vs Specificity?

Accuracy: 0.5

358400×3 Matrix{Float64}:
      1.0  1.0  1.0
      2.0  1.0  1.0
      3.0  1.0  1.0
      4.0  1.0  1.0
      5.0  1.0  1.0
      6.0  1.0  1.0
      7.0  1.0  1.0
      8.0  1.0  1.0
      9.0  1.0  1.0
     10.0  1.0  1.0
     11.0  1.0  1.0
     12.0  1.0  1.0
     13.0  1.0  1.0
      ⋮         
 358389.0  1.0  0.0
 358390.0  1.0  0.0
 358391.0  1.0  0.0
 358392.0  1.0  0.0
 358393.0  1.0  0.0
 358394.0  1.0  0.0
 358395.0  1.0  0.0
 358396.0  1.0  0.0
 358397.0  1.0  0.0
 358398.0  1.0  0.0
 358399.0  1.0  0.0
 358400.0  1.0  0.0

In [8]:
# Change up the layers
# Try Min-max normalisation
# Try 10-fold